In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dataset=pd.read_csv("../input/sms-spam-collection-dataset/spam.csv",encoding='latin-1')


In [ ]:
dataset['Unnamed: 2'].fillna("",inplace=True)
dataset['Unnamed: 3'].fillna("",inplace=True)
dataset['Unnamed: 4'].fillna("",inplace=True)
dataset['new']=dataset['v2']+dataset['Unnamed: 2']+dataset['Unnamed: 3']+dataset['Unnamed: 4']
dataset['v1']=(dataset['v1']=="spam").astype(int)
dataset.drop(['v2','Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1,inplace=True)
dataset.head()

In [ ]:
from imblearn.over_sampling import RandomOverSampler
import collections
ros = RandomOverSampler(random_state=0)
X, y = ros.fit_resample(dataset[dataset.columns.drop(['v1'])], dataset['v1'])

print(collections.Counter(dataset['v1']),collections.Counter(y))
X=pd.DataFrame({'text': X.iloc[:, 0]})


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
training_sentences = X['text'].values
tokenizer = Tokenizer()
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
training_sequences = tokenizer.texts_to_sequences(training_sentences)
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
def sequence_to_text(list_of_indices):
    # Looking up words in dictionary
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return(words)

# Creating texts 
my_texts = list(map(sequence_to_text, training_sequences))


In [ ]:
j=[]
for i in range(len(my_texts)):
  j.append(str(' '.join(my_texts[i])))
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000)
x = cv.fit_transform(j).toarray()


In [ ]:
from sklearn.model_selection import train_test_split

trainx,testx,trainy,testy = train_test_split(x,y,test_size=0.2,random_state=2)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf_model = RandomForestClassifier(n_estimators=1000,max_depth=100,random_state=1,max_features=1)
rf_model.fit(trainx, trainy)
rf_test_predictions = rf_model.predict(testx)
rf_test_mae = accuracy_score(rf_test_predictions, testy)
print(rf_test_mae)

In [ ]:
from sklearn import svm
svmclf_model = svm.SVC()
svmclf_model.fit(trainx, trainy)
svc = svmclf_model.predict(testx)
svm_mae = accuracy_score(svc, testy)
print(svm_mae)

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(random_state=0,max_iter=1000)
logreg.fit(trainx, trainy)
logregpred = logreg.predict(testx)
logreg_mae = accuracy_score(logregpred, testy)
print(logreg_mae)

In [ ]:
from xgboost import XGBClassifier

my_model_1 = XGBClassifier(random_state=0)

my_model_1.fit(trainx, trainy)
xgpredictions=my_model_1.predict(testx)
print(accuracy_score(xgpredictions, testy))


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(testy, logregpred),display_labels=['Ham','Spam'])
disp.plot()

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(testy, logregpred)
plt.plot(fpr,tpr)